In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scripts.data_processing import handle_missing_data, encode_categorical_data, feature_engineering
from scripts.modeling import evaluate_model, plot_feature_importance
import shap


In [ ]:

# Load the dataset
file_path = '../data/MachineLearningRating_v3.txt'
df = pd.read_csv(file_path, delimiter='|')



In [ ]:

# Data Preparation
# Step 1: Handle missing data
df_cleaned = handle_missing_data(df)


In [ ]:

# Step 2: Feature Engineering (if necessary)
df_cleaned = feature_engineering(df_cleaned)



In [ ]:
# Step 3: Encode categorical variables
df_encoded = encode_categorical_data(df_cleaned)



In [ ]:
# Step 4: Train-test split
X = df_encoded.drop(columns=['TotalClaims'])  # Features
y = df_encoded['TotalClaims']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Standardize features for Linear Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [ ]:
# Model 1: Linear Regression
print("Training Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)
print("Linear Regression Performance:")
evaluate_model(y_test, y_pred_lr)


In [ ]:

# Model 2: Decision Tree
print("Training Decision Tree...")
tree_model = DecisionTreeRegressor(random_state=42, max_depth=10)  # Limit depth to avoid overfitting
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)
print("Decision Tree Performance:")
evaluate_model(y_test, y_pred_tree)


# Sample a subset of data if the dataset is too large
df_sampled = df_encoded.sample(frac=0.1, random_state=42)  # Use 10% of the data
X = df_sampled.drop(columns=['TotalClaims'])  # Features
y = df_sampled['TotalClaims']  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Random Forest Model (reduced n_estimators and no max_depth)
print("Training Random Forest...")
rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Performance:")
evaluate_model(y_test, y_pred_rf)


# Model 4: XGBoost
# Train-test split
X = df_encoded.drop(columns=['TotalClaims'])  # Features
y = df_encoded['TotalClaims']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the datasets into DMatrix, required by XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.1,
    'max_depth': 5,
    'random_state': 42,
    'eval_metric': 'rmse'  # Root Mean Squared Error
}

# Train the XGBoost model with early stopping
eval_set = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model = xgb.train(params, dtrain, num_boost_round=100, evals=eval_set, early_stopping_rounds=10, verbose_eval=False)

# Predict on the test set
y_pred_xgb = xgb_model.predict(dtest)

# Evaluate the model performance
print("XGBoost Performance:")
evaluate_model(y_test, y_pred_xgb)

print("Feature Importance Analysis using SHAP...")
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_train)


In [ ]:

# Plot SHAP summary plot
shap.summary_plot(shap_values, X_train)

# Plot feature importance for Random Forest
plot_feature_importance(rf_model, X_train)